In [24]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xarray
import cartopy.crs as ccrs
from scipy.signal import detrend
#import eofs

In [25]:
pfad = "/home/srvx11/lehre/users/a1127897/ipython/Klima2/Data/mslp_daily/"

In [26]:
def open_df(pfad):
    df = xr.open_mfdataset(pfad+'*.nc',chunks={'initial_time0_hours':504,'g0_lat_1':29,'g0_lon_2':29})
    df = df.rename({'initial_time0_hours':'time','g0_lat_1':'lat','g0_lon_2':'lon','PRMSL_GDS0_MSL':'mslp'}).drop("initial_time0_encoded")
    df.coords['lon'] = (df.coords['lon'] + 180) % 360 - 180
    return df
def mean_std(df):
    df_mean = df.resample(time='1D').mean().chunk({'time':504,'lat':29,'lon':29})
    df_roll_mean = df_mean.rolling(time=20,center=True).mean().dropna('time').groupby('time.dayofyear').mean(axis=0)
    
    new_dim = df_mean.rolling(time=20, center=True).construct('dim_roll_days') #Die +- 10 Tage in eine eigene Dim stecken
    stacked_dim =new_dim.stack(line=('lat','lon')) # Stack into Dimension lat, lon
    df_std = stacked_dim.groupby('time.dayofyear').std() #Std der 20 Tage pro Tag des Jahres berechnen
    return df_mean,df_std
def anomaly_nor(df,mean,std):
    anomaly = df - mean
    anomaly_norm = anomaly / std
    return anomaly,anomaly_norm

In [30]:
df = open_df(pfad)
df_mean,df_std = mean_std(df)
df_ano, df_norm_ano = anomaly_nor(df,df_mean,df_std)

/opt/anaconda3/lib/python3.7/site-packages/xarray/core/groupby.py:758: FutureWarning: Default reduction dimension will be changed to the grouped dimension in a future version of xarray. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  allow_lazy=True, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/dask/array/core.py:3233: PerformanceWarning: Increasing number of chunks by factor of 25
  **blockwise_kwargs)


In [31]:
df_norm_ano

<xarray.Dataset>
Dimensions:    (dayofyear: 366, lat: 29, lon: 29, time: 731)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2016-12-31
  * lat        (lat) float32 67.5 66.25 65.0 63.75 ... 36.25 35.0 33.75 32.5
  * lon        (lon) float32 -10.0 -8.75 -7.5 -6.25 ... 21.25 22.5 23.75 25.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    mslp       (time, lat, lon, dayofyear) float32 dask.array<shape=(731, 29, 29, 366), chunksize=(31, 29, 29, 1)>